<a href="https://colab.research.google.com/github/ella00100/RenewableEnergyModel/blob/main/renewableEnergy_Model_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##재생에너지 발전량 예측제도

In [1]:
from google.colab import files
uploaded = files.upload()

Saving test_data.csv to test_data.csv


In [6]:
import io
import pandas as pd
data = pd.read_csv('test_data.csv')

In [7]:
data.head()

,일자,구분1,발전소코드,발전소용량(KW),0:00,1:00,2:00,3:00,4:00,5:00,...,15:00,16:00,17:00,18:00,19:00,20:00,21:00,22:00,23:00,합계
0,2023-03-01,실측,G1053,249.48,0.00,0.00,0,0,0,0.00,...,0.10,65.80,33.60,133.70,161.1,150.5,0,0,0,640.90
1,2023-03-01,예측,G1053,249.48,0.95,0.97,0.94,0.97,0.96,0.95,...,20.08,93.61,35.62,161.20,161.2,161.2,46.95,0.89,0.98,739.26
2,2023-03-01,실측,G1018,288.00,0.00,0.00,0,0,0,0.00,...,101.26,53.50,29.09,2.21,0.0,0,0,0,0,787.25
3,2023-03-01,예측,G1018,288.00,0.00,0.00,0,0,0,0.00,...,93.33,30.84,27.05,1.43,0.0,0,0,0,0,618.75
4,2023-03-01,실측,G1034,298.08,12.40,0.00,0,0,0,0.00,...,18.60,63.80,28.80,0.60,0.0,164.2,135.6,0,0,552.40


In [22]:
print(data.dtypes)

일자            object
구분1           object
발전소코드         object
발전소용량(KW)    float64
0:00         float64
1:00         float64
2:00          object
3:00          object
4:00          object
5:00         float64
6:00         float64
7:00         float64
8:00         float64
9:00         float64
10:00        float64
11:00        float64
12:00        float64
13:00        float64
14:00        float64
15:00        float64
16:00        float64
17:00        float64
18:00        float64
19:00        float64
20:00         object
21:00         object
22:00         object
23:00         object
합계           float64
dtype: object


In [8]:
data['발전소코드'].unique()

array(['G1053', 'G1018', 'G1034', 'S5004', 'S5043', 'G1039', 'S5007',
       'S5020', 'G1022', 'S5014', 'G1019', 'S5027', 'S5017', 'G1032',
       'S5021', 'S5024', 'S5032', 'S5002', 'S5028', 'S5038', 'S5009',
       'S5018', 'G1066', 'S5026', 'S5030', 'G1026', 'G1025', 'G1021',
       'G1060', 'G1030', 'S5005', 'G1017', 'S5025', 'S5033', 'G1063',
       'S5029', 'G1059', 'S5016', 'S5015', 'S5006', 'S5010', 'G1020',
       'S5041', 'S5047', 'S5022', 'S5019', 'S5008', 'S5040', 'S5011',
       'S5039', 'S5046', 'G1065', 'G1023', 'S5003', 'S5023', 'G1024',
       'G1061', 'S5042', 'S5050', 'G1051', 'G1054', 'S5001', 'G1062',
       'S5013', 'G1055', 'G1031', 'G1028', 'G1027', 'G1029', 'G1067',
       'G1068', 'G1064', 'G1058', 'G1057', 'G1056', 'G1052', 'G1050',
       'S5048', 'S5049', 'G1033', 'S5012', 'S5035', 'S5031', 'S5037',
       'S5034', 'S5045', 'S5036', 'S5044', 'B1739', 'B1737', 'B1742',
       'B1740', 'B1738', 'B1741', 'B1788', 'B1786', 'B1787', 'B1780',
       'B1779', 'B17

In [9]:
data['구분1'].unique()

array(['실측', '예측'], dtype=object)

In [13]:
code_first = data['발전소코드'].str[0]
code_first.unique()

array(['G', 'S', 'B', 'N', 'M', 'V'], dtype=object)

 - 'G'는 화력 발전소
 - 'S'는 태양광 발전소
 - 'B'는 바이오매스 발전소
 - 'N'은 원자력 발전소
 - 'M'은 연료전지 발전소
 - 'V'는 풍력 발전소

In [16]:
min_size = min(data['발전소용량(KW)'])
max_size = max(data['발전소용량(KW)'])

print(min_size, max_size)

98.825 5587.56


In [17]:
real_data = data.iloc[::2]
predic_data = data.iloc[1::2]

In [18]:
real_data.head()

,일자,구분1,발전소코드,발전소용량(KW),0:00,1:00,2:00,3:00,4:00,5:00,...,15:00,16:00,17:00,18:00,19:00,20:00,21:00,22:00,23:00,합계
0,2023-03-01,실측,G1053,249.48,0.0,0.0,0,0,0,0.0,...,0.10,65.80,33.60,133.70,161.1,150.5,0,0,0,640.90
2,2023-03-01,실측,G1018,288.00,0.0,0.0,0,0,0,0.0,...,101.26,53.50,29.09,2.21,0.0,0,0,0,0,787.25
4,2023-03-01,실측,G1034,298.08,12.4,0.0,0,0,0,0.0,...,18.60,63.80,28.80,0.60,0.0,164.2,135.6,0,0,552.40
6,2023-03-01,실측,S5004,300.80,0.0,0.0,0,0,0,0.0,...,100.80,28.18,22.52,3.54,0.0,0,0,0,0,575.38
8,2023-03-01,실측,S5043,320.76,0.0,0.0,0,0,0,0.0,...,202.84,240.25,236.49,195.64,192.6,42.67,0,0,0,1951.65


In [19]:
predic_data.head()

,일자,구분1,발전소코드,발전소용량(KW),0:00,1:00,2:00,3:00,4:00,5:00,...,15:00,16:00,17:00,18:00,19:00,20:00,21:00,22:00,23:00,합계
1,2023-03-01,예측,G1053,249.48,0.95,0.97,0.94,0.97,0.96,0.95,...,20.08,93.61,35.62,161.20,161.20,161.2,46.95,0.89,0.98,739.26
3,2023-03-01,예측,G1018,288.00,0.00,0.00,0,0,0,0.00,...,93.33,30.84,27.05,1.43,0.00,0,0,0,0,618.75
5,2023-03-01,예측,G1034,298.08,0.85,0.86,0.85,0.86,0.85,0.86,...,89.61,66.66,25.73,1.62,0.88,172.6,172.6,83.4,1.57,685.61
7,2023-03-01,예측,S5004,300.80,0.00,0.00,0,0,0,0.00,...,124.39,81.26,17.75,1.33,0.00,0,0,0,0,758.24
9,2023-03-01,예측,S5043,320.76,5.40,5.24,5.03,5.05,5.09,4.87,...,199.96,131.53,190.43,190.43,190.43,133.29,4.15,4.9,5.16,1170.54


In [24]:
data['일자'] = pd.to_datetime(data['일자'])

In [26]:
for col in data.columns:
    if data[col].dtype == 'object':
        data[col] = pd.to_numeric(data[col], errors='coerce')

In [27]:
data.dtypes

일자           datetime64[ns]
구분1                 float64
발전소코드               float64
발전소용량(KW)           float64
0:00                float64
1:00                float64
2:00                float64
3:00                float64
4:00                float64
5:00                float64
6:00                float64
7:00                float64
8:00                float64
9:00                float64
10:00               float64
11:00               float64
12:00               float64
13:00               float64
14:00               float64
15:00               float64
16:00               float64
17:00               float64
18:00               float64
19:00               float64
20:00               float64
21:00               float64
22:00               float64
23:00               float64
합계                  float64
dtype: object

In [28]:
time_names = [f"{i}:00" for i in range(24)]

time_means = []
for col in time_names:
    if col in data.columns:
        time_means.append(data[col].mean())
    else:
        time_means.append(None)

for col, col_mean in zip(time_names, time_means):
    print(f"time {col}: Mean = {col_mean}")

time 0:00: Mean = 5.535597442259973
time 1:00: Mean = 3.8478946363809885
time 2:00: Mean = 1.7839247876300468
time 3:00: Mean = 1.4298224513172966
time 4:00: Mean = 1.4120607044001146
time 5:00: Mean = 1.3956155754915058
time 6:00: Mean = 3.0692231341859135
time 7:00: Mean = 60.7091658713495
time 8:00: Mean = 230.75691353311706
time 9:00: Mean = 433.8397766749379
time 10:00: Mean = 313.12604695552585
time 11:00: Mean = 350.99882420309217
time 12:00: Mean = 374.8751355220462
time 13:00: Mean = 401.33859706050777
time 14:00: Mean = 520.1749933193357
time 15:00: Mean = 478.6058484443596
time 16:00: Mean = 334.0190332124451
time 17:00: Mean = 291.7138242030922
time 18:00: Mean = 267.84938919641155
time 19:00: Mean = 266.22490837946174
time 20:00: Mean = 299.4336012217238
time 21:00: Mean = 243.33494081710575
time 22:00: Mean = 114.37719289873056
time 23:00: Mean = 41.38875536890331


In [32]:
start = min(data['일자'])
end = max(data['일자'])

print(start, end)

2023-03-01 00:00:00 2023-03-31 00:00:00
